#### 1. Install Dependencies

In [ ]:
pip install -r ../requirements.txt -q

In [ ]:
pip show pinecone-client

#### 2. Verify python-dotenv

In [ ]:
import os
import pinecone
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

#### 3. Split Text Into Chunks

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

with open('nehru_speech.txt') as f:
  nehru_speech = f.read()
  
text_splitter = RecursiveCharacterTextSplitter(
  chunk_size = 100,
  chunk_overlap = 20,
  length_function = len,
)

chunks = text_splitter.create_documents([nehru_speech])
print(f'total chunks: {len(chunks)}')
print(chunks[0])
print(chunks[0].page_content)

#### 4. Embedding Cost

In [ ]:
def get_embedding_cost(texts):
  import tiktoken
  enc = tiktoken.encoding_for_model('text-embedding-ada-002')
  total_tokens = sum([len(enc.encode(page.page_content)) for page in texts])
  print(f'total tokens: {total_tokens}')
  print(f'embedding cost USD: {total_tokens / 1000 * 0.0004:.6f}')
  
get_embedding_cost(chunks)

#### 5. Create Embeddings

In [ ]:
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

# vector = embeddings.embed_query('abcd')
# print(vector)

vector = embeddings.embed_query(chunks[0].page_content)
print(vector)

#### 6. Insert Embeddings Into Pinecone Index

In [ ]:
import os
import pinecone
from langchain.vectorstores import Pinecone

pinecone.init(
  api_key=os.environ.get("PINECONE_API_KEY"),
  environment=os.environ.get("PINECONE_ENV")
)

# delete all indexex (only 1 index is allowed per account in free tier)
print('deleting all indexes...')
for idx_name in pinecone.list_indexes():
  pinecone.delete_index(idx_name)  
print('indexes deleted')

# create index
idx_name='nehru-speech'
print(f'creating index {idx_name}...')
pinecone.create_index(name=idx_name, dimension=1536, metric='cosine')
print('index created')

# creates a vector store by taking chunks as inputs
# then creating embeddings for it using OpenAIEmbeddings
# then stroing it in our specified index
print(f'adding {len(chunks)} vectors to index {idx_name} for {len(chunks)} chunks...')
vector_store = Pinecone.from_documents(chunks, embeddings, index_name = idx_name)
print(f'vectors added')

#### 7. Asking Questions (Similarity Search)

In [ ]:
# these queries simply return the exact chunk as stored in bector db based on similarity vector
# the results are kinda accurate but still not how a human would answer
# we need the answers in a natural language

print('-' * 10)
query = 'What is the speech about?'
result = vector_store.similarity_search(query)
print(f'query: {query}')
for i in range(len(result)):
  print(f'result {i} : {result[i].page_content}')

print('-' * 10)
query = 'What will happen at stroke of midnight?'
result = vector_store.similarity_search(query)
print(f'query: {query}')
for i in range(len(result)):
  print(f'result {i} : {result[i].page_content}')
  
print('-' * 10)
query = 'What is the ambitious man trying to achieve?'
result = vector_store.similarity_search(query)
print(f'query: {query}')
for i in range(len(result)):
  print(f'result {i} : {result[i].page_content}')

#### 8. Asking Same Questions to LLM

In [ ]:
# fix the disadvantages of previous approach
# we use the answers from vector store & feed them to LLM

from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

# init chat based llm
llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=1.0)

# retriever makes it easy to combine docsuments with llms
retriever = vector_store.as_retriever(search_type='similarity', search_kwargs={'k': 3})

# create a chain that combines llm & retriever
# chain_type='stuff' is a predefined chain that uses all text from documents
chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever, chain_type='stuff')

print('-' * 10)
query = 'What is the speech about?'
result = chain.run(query)
print(f'query: {query}')
print(f'result: {result}')

print('-' * 10)
query = 'What will happen at stroke of midnight?'
result = chain.run(query)
print(f'query: {query}')
print(f'result: {result}')

print('-' * 10)
query = 'What is the ambitious man trying to achieve?'
result = chain.run(query)
print(f'query: {query}')
print(f'result: {result}')